In [1]:
from osgeo import gdal
import numpy as np
import pandas as pd
import richdem as rd
import os
import glob

Constantes

In [10]:
DATA_PATHS = [
    './results/data_merged_all_gt_map_0.csv',
    './results/data_merged_all_gt_map_1.csv',
    './results/data_merged_all_gt_map_2.csv',
    './results/data_merged_all_gt_map_3.csv',
    './results/data_merged_all_gt_map_4.csv',
    './results/data_merged_all_gt_map_5.csv',
    './results/data_merged_all_gt_map_6.csv',
    './results/data_merged_all_gt_map_7.csv',
    './results/data_merged_all_gt_map_8.csv',
    './results/data_merged_all_gt_map_9.csv',
    './results/data_merged_all_gt_map_10.csv'
]
RES_DATA = './results/data_merged_all_gt_map_final_23_10.csv'

Graficar la data

In [3]:
class DefaultMetaObj:
    geotransform = [0,1,0,0,0,-1]

class DataGenerator:
    def __init__(self, data_directories):
        self.data_directories = data_directories

    def getData(self):
        output = pd.DataFrame()
        for directory in self.data_directories:
            output = pd.concat([output, pd.read_csv(directory, index_col=0)])
        return output

In [4]:
dg = DataGenerator(DATA_PATHS)
data = dg.getData()

In [5]:
len(data)

106123

In [6]:
insivumeh_columns = [
    'rock_geo','rock_period','fis_nomr','rock_type',
    'soil_rec_symbol','soil_rec_mat_origin','soil_rec_sew_class',
    'soil_rec_tex_sup','soil_rec_c_tex_sup','soil_rec_tex_sub',
    'soil_rec_c__tex_sub','soil_rec_risk_eros','soil_tax','soil_use'
]

for col in insivumeh_columns:
    if '-999' in data[col].value_counts():
        data.loc[data[col] == '-999', col] = pd.NA

In [7]:
insivumeh_columns2 = [
    'soil_rec_mat_origin','soil_rec_sew_class',
    'soil_rec_tex_sup','soil_rec_c_tex_sup','soil_rec_tex_sub',
    'soil_rec_c__tex_sub','soil_rec_risk_eros','soil_tax','soil_use'
]
for col in insivumeh_columns:
        data.loc[data['soil_rec_symbol'] == 'AGUA', col] = 'AGUA'

In [8]:
na_data = data.isnull().sum()
na_data[na_data > 0]

rock_geo                        1105
rock_period                     1107
fis_nomr                        1107
rock_type                       1233
soil_rec_symbol                 1105
soil_rec_mat_origin             4119
soil_rec_sew_class              4899
soil_rec_tex_sup                4952
soil_rec_c_tex_sup              4952
soil_rec_tex_sub                5697
soil_rec_c__tex_sub             7992
soil_rec_risk_eros              5489
soil_tax                        1055
soil_use                        1105
precipitation_mm_last_7_days       9
dtype: int64

In [20]:
data.dropna(subset=[*insivumeh_columns, 'precipitation_mm_last_7_days'], inplace=True)

In [21]:
na_data = data.isnull().sum()
na_data[na_data > 0]

Series([], dtype: int64)

In [9]:
len(data)

106123

In [24]:
data['landslide'].value_counts()

In [ ]:
data2 = data
data2['event_date'] = pd.to_datetime(data2['event_date'])

In [ ]:
sum = 0
for year in range(2008,2023):
    count = data2[data2.event_date.dt.year==year].count()
    print(year,count['landslide'])
    if year >= 2015:
        sum += count['landslide']
print(sum)

2008 168
2009 16
2010 653
2011 174
2012 63
2013 58
2014 229
2015 82
2016 373
2017 545
2018 426
2019 500
2020 1242
2021 555
2022 147
3870


In [ ]:
data2[data2.event_date.dt.year > 2015]['landslide'].value_counts()

0    2213
1    1575
Name: landslide, dtype: int64

In [11]:
data.to_csv(RES_DATA)  